In [ ]:
# 1.1 安裝 Azure CLI
!curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash

In [ ]:
# 1.2 登入 Azure
!az login

In [ ]:
# 1.3 相關資訊設定
# 設定資源群組所在位置
resource_group_location="westus"

# 設定資源群組名稱
date="$(date -d '8hour' +'%Y%m%d')"
resource_group="test"+date

# 設定 IoT Hub 名稱
tmp=!cat /sys/class/net/eth0/address | sed -e "s/://g"
hub_name="for-rpi-"+date+tmp[0]

# 設定 Device ID
device_id="rpi-edge-no-1"

# 設定儲存帳號
storage_account = "fileupload" + date

In [ ]:
# 1.4 新建資源群組
!az group create -l $resource_group_location -n $resource_group

In [ ]:
# 1.5 建立 IoT Hub
!az iot hub create --resource-group $resource_group --name $hub_name --sku S1

In [ ]:
# 1.6 建立儲存帳號
!az storage account create -n $storage_account -g $resource_group

In [ ]:
# 1.7 取得 Azure Storage Account 的 Connection String
storage_account_conn_str = !az storage account show-connection-string -n $storage_account \
-g $resource_group | grep -o '"connectionString": "[^"]*' | grep -o '[^"]*$'
!echo $storage_account_conn_str

In [ ]:
# 1.8 建立 Storage Container
container_name = "edge"
!az storage container create --name $container_name \
--connection-string $storage_account_conn_str

In [ ]:
# 1.9 取得 Azure Storage Account 的 Primary Key 的 Connection String
primary_key = !az storage account show-connection-string -g $resource_group \
-n $storage_account --key primary | grep -o '"connectionString": "[^"]*' | grep -o '[^"]*$'
!echo $primary_key

In [ ]:
# 1.10 建立 Storage account Access key 的 Connection string
key = !az storage account keys list --account-name $storage_account -g $resource_group \
 | grep -m 1 -o '"value": "[^"]*' | grep -o '[^"]*$'
keyConnectionString = 'DefaultEndpointsProtocol=https' + '\;AccountName=' + storage_account + \
'\;AccountKey=' + key[0] + \
'\;EndpointSuffix=core.windows.net'

In [ ]:
# 1.11 設定 Azure IoT Hub 的 Storage Container 與 File Notification
!az iot hub update --resource-group $resource_group --name $hub_name \
--fileupload-storage-container-name $container_name \
--fileupload-notifications true \
--fileupload-storage-connectionstring $keyConnectionString

In [ ]:
# 1.12 建立 IoT Edge
!az iot hub device-identity create --hub-name $hub_name --device-id $device_id

In [ ]:
# 1.13 取得 Azure IoT Hub 的 Connection String
!az iot hub show-connection-string --resource-group $resource_group --hub-name $hub_name \
| grep -o '"connectionString": "[^"]*' | grep -o '[^"]*$'

In [ ]:
# 1.14 取得 Azure Device 的 Connection String
!az iot hub device-identity show-connection-string --resource-group $resource_group \
--device-id $device_id --hub-name $hub_name | grep -o '"connectionString": "[^"]*' | \
grep -o '[^"]*$'